In [1]:
import pyspark

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
import re

stop_words = ["a", "about", "above", "after", "all", "am", "an", "and", "any",\
                "are", "as", "at", "be", "because", "been", "before", "being",\
              "below", "tween", "bot", "by", "could", "did", "do", "does", "doing",\
              "down", "during", "each", "few", "for", "from", "further",\
              "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her",\
              "here", "here's", "hers", "herself", "him", "himself", "his", "how",\
              "how's", "i", "i'd", "i'll", "i'm", "i've", "u'if", "in", "into", "is",\
              "it", "it's", "its", "itself", "let's", "me", "more", "most",\
              "my", "myself", "of", "off", "on", "once",\
              "only", "or", "other", "ought", "our", "ours", "ourselves", "out",\
              "over", "own", "same", "she", "she'd", "she'll", "she's",\
              "should", "so", "some", "such", "than", "that","that's",\
              "the", "their", "theirs", "them", "themselves", "then", "there", "there's",\
              "these", "they", "they'd", "they'll", "they're", "they've", "this",\
              "those", "through", "to", "too", "under", "until", "up", "very", "was",\
             "we", "we'd", "we'll", "we're", "we've", "were",\
              "what", "what's", "when", "when's", "where", "where's", "which", "while",\
              "who", "who's", "whom", "why", "why's", "with", "would",\
              "you", "you'd", "you'll", "you're", "you've", "your", "yours",\
              "yourself", "yourselves"]

stop_words = set(stop_words)

palabras_sentimiento_inverso = ["not", "ain't", "bareley", "aren't", "can't", "couldn't", "didn't", "doesn't",\
                                "don't", "hadn't", "few", "hardly", "low", "merely", "neither", "never",\
                                "no", "nor", "nobody", "none", "nope", "nothing", "rarely", "seldom",\
                                "hasn't", "haven't", "isn't", "mustn't", "shan't", "shouldn't", "used",\
                                "wasn't", "weren't", "won't", "wouldn't", "zero"]


palabras_sentimiento_inverso = set(palabras_sentimiento_inverso)



def quitar_reviews_repetidos(reviewsRDD):
    reviews_sin_rep = reviewsRDD.map( lambda review: (review.Text, review) )\
                    .reduceByKey( lambda review1, review2: review1 )

    reviews_sin_rep = reviews_sin_rep.map(lambda x: x[1])

    return reviews_sin_rep

def preprocesar(review):
    texto = quitar_signos(review.Text)
    resumen = quitar_signos(review.Summary)

#    texto = texto.lower()
#    resumen = resumen.lower()

    texto_palabras = texto.split(" ")
    resumen_palabras = resumen.split(" ")

#    texto_palabras = texto_palabras + ngrams(texto_palabras, 2)
#    resumen_palabras = resumen_palabras + ngrams(resumen_palabras, 2)

    texto_palabras = quitar_stopWords(texto_palabras)
    resumen_palabras = quitar_stopWords(resumen_palabras)

    estructura = [review.Id, review.Prediction, resumen_palabras+texto_palabras]

    return estructura


def quitar_signos(texto):
    text = ""
    if texto is not None:
        text = texto.replace("<br />", " ")
        text = re.sub("[^a-zA-Z' ]", ' ', text)

    return text


def quitar_stopWords(texto):
    text = []

    for i in range( len(texto) ):
        palabra = texto[i].lower()
        if palabra not in stop_words and palabra != '':

            if palabra in palabras_sentimiento_inverso and i+1 < len(texto):
                palabra = palabra + " " + texto[i+1].lower()
                text.append(palabra)

            if i > 0 and texto[i-1].lower() not in palabras_sentimiento_inverso:
                text.append(palabra)

    return text


def ngrams(texto, n):
    #texto = texto.split(" ")
    lista_ngrams = []
    for i in xrange(0, len(texto)-n-1):
        lista_ngrams.append(" ".join(texto[i:i+n]))

    return lista_ngrams


In [3]:
import pyspark_csv as pycsv

trainRDD = sc.textFile('data_train.csv')
train_dataframe = pycsv.csvToDataFrame(sqlCtx, trainRDD, parseDate=False)
train = train_dataframe.rdd


train_sin_rep = quitar_reviews_repetidos(train)

train_preprocesado = train_sin_rep.map(preprocesar)

for x in train_preprocesado.take(1):
    for y in x:
        print y


279582
5
[u'healthy', u'delicious', u'tastes', u'great', u'spread', u'apple', u'banana', u'honey', u'adds', u'just', u'enough', u'sweetness', u'make', u'taste', u'like', u'guilty', u'pleasure', u'without', u'making', u'overly', u'sugary']


In [4]:
testRDD = sc.textFile('data_test.csv')
test_dataframe = pycsv.csvToDataFrame(sqlCtx, testRDD, parseDate=False)
test = test_dataframe.rdd


test_sin_rep = quitar_reviews_repetidos(test)

test_preprocesado = test_sin_rep.map(preprocesar)

for x in test_preprocesado.take(1):
#    print "x: ", x
    for y in x:
        print y


201541
3
[u'not entirely', u'satisfied', u'though', u'shipping', u'super', u'fast', u'not as', u'not as', u'good', u'hoped', u'pretty', u'sure', u'received', u'pound', u'half', u'definitely', u'not the', u'not the', u'full', u'two', u'pounds', u'guess', u'fine', u"don't see", u"don't see", u'finishing', u'given', u'flavor', u'decent', u'but', u'no where', u'no where', u'near', u'good', u'dried', u'papaya', u'can', u"didn't seem", u"didn't seem", u'fresh', u'either', u'probably', u'wont', u'reordering', u'item', u'again']


In [5]:
## SVM DeltaTFIDF

def bow(review):
    texto = review[2]
    frec = {}
    for word in texto:
        if word in frec:
            frec[word] += 1
        else:
            frec[word] = 1

    review.append(frec)
    return review
#    return frec

train_frec = train_preprocesado.map(bow)


def frec_por_doc(review):
    texto = set(review[2])
    words = []
    for word in texto:
        words.append( (word, 1) )

    return words


In [6]:

positivas = train_preprocesado.filter( lambda review: review[1]==4 or review[1]==5)
cant_pos = positivas.count()

frec_por_doc_pos = positivas.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frec_por_doc_pos = frec_por_doc_pos.collect()

frec_pos = {}

for word in frec_por_doc_pos:
    frec_pos[word[0]] = word[1]


negativas = train_preprocesado.filter( lambda review: review[1]==1 or review[1]==2 or review[1]==3 )
cant_neg = negativas.count()

frec_por_doc_neg = negativas.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frec_por_doc_neg = frec_por_doc_neg.collect()

frec_neg = {}

for word in frec_por_doc_neg:
    frec_neg[word[0]] = word[1]


In [7]:
import math
from math import sqrt


def delta_tfidf_ant(review_test, review_train):
    texto = review_test[2]
    frec_word = review_train[3]
    Vt = 0

    ##TF-IDF
    k = 2
    b = 0.75
    long_doc = len(texto)
    norm = 1 - b + b * (long_doc / promedio_long_docs)

    for word in texto:
        ##TF-IDF
        r = 0
        if word in frec_word:
            tf = ( (k+1)*frec_word[word] ) / ( frec_word[word] + k*norm )
            idf = math.log( float(cant_reviews+1)/frec[word] )
            r = tf * idf
#            r = frec_word[word]

        if word in frec_pos and word in frec_neg:
            Pt = frec_pos[word]
            Nt = frec_neg[word]

#            Vt += r * math.log(float(Nt)/Pt , 2 )
            Vt += r * float(Nt)/Pt

        else:
            Vt += 0
    """
        pos = 0
        if word in frec_pos:
            Pt = frec_pos[word]
            pos = r * math.log( cant_pos/float(Pt) , 2 )

        neg = 0
        if word in frec_neg:
            Nt = frec_neg[word]
            neg = r * math.log( cant_neg/float(Nt) , 2 )

        Vt += pos - neg
    """
    return [Vt, review_train[1]]


In [8]:

def delta_tfidf(review_test, review_train):
    texto = review_test[2]
    frec_word = review_train[3]

    r = 0
    k = 1.2
    # NORMALIZAR
    b = 0.75
    long_doc = len(texto)
    norm = 1 - b + b * (long_doc / promedio_long_docs)

    for word in texto:
        ## TF-IDF
        tf = 0

        if word in frec_word:
            #BM25
            tf = ( (k+1)*frec_word[word] ) / ( frec_word[word] + k*norm )

        idf_pos = 0
        if word in frec_pos:
            Pt = frec_pos[word]
            idf_pos = math.log( 1 + (cant_pos+1) / Pt)

        idf_neg = 0
        if word in frec_neg:
            Nt = frec_neg[word]
            idf_neg = math.log( 1 + (cant_neg+1) / Nt)

        r += tf * ( idf_pos - idf_neg )

    return [abs(r), review_train[1]]


In [9]:
## TF-IDF
import math


frecs_por_docs = train_preprocesado.flatMap(frec_por_doc).reduceByKey( lambda x,y: x+y )
frecs_por_docs = frecs_por_docs.collect()

frec = {}

for word in frecs_por_docs:
    frec[word[0]] = word[1]

#print " "
#print "frecs_por_docs: ", frec


cant_reviews = train_preprocesado.count()


In [10]:
## NORMALIZAR DATOS

train_long_docs = train_preprocesado.map( lambda review: (1, len(review[2])) ).reduceByKey( lambda x, y: x+y )
long_docs = train_long_docs.collect()
long_docs = long_docs[0][1]
print "long: ", long_docs

promedio_long_docs = float(long_docs) / cant_reviews
print "promedio: ", promedio_long_docs



long:  10316425
promedio:  45.0347481415


In [11]:

def sentimiento(word, word_ant):
    sentimiento = 0

    if word in palabras_pos or word in palabras_neg:
        return 2

    return 1

"""
    if word in palabras_pos:
        if word_ant not in palabras_sentimiento_inverso:
            sentimiento += palabras_pos[word]
        else:
            sentimiento += palabras_pos[word]

    if word in palabras_neg:
        if word_ant not in palabras_sentimiento_inverso:
            sentimiento -= palabras_neg[word]
        else:
            sentimiento -= palabras_neg[word]

    return sentimiento
"""

def tfidf(review_test, review_train):
    texto = review_test[2]
    frec_word = review_train[3]
    r = 0

    k = 1.2
    b = 0.75
    long_doc = len(texto)
    norm = 1 - b + b * (long_doc / promedio_long_docs)

    ## SENTIMIENTO
    for i in range( len(texto) ):
        word_ant = ""
        if i>0:
            word_ant = texto[i-1]
        word = texto[i]
#        sent = sentimiento(word, word_ant)

        ## TF-IDF
        tf = 0
        idf = 0

        if word in frec_word:
            #BM25
            tf = ( (k+1)*frec_word[word] ) / ( frec_word[word] + k*norm )
#            print "cant_reviews: ", cant_reviews
#            print "frec[word]: ", frec[word]
            idf = math.log( float(cant_reviews+1)/frec[word] )

        r += tf * idf
#        r += tf * idf * sent

    return [r, review_train[1]]


In [12]:

def knn(texto_estruct, k):

#    dist = train_frec.map(lambda vecino: tfidf(texto_estruct, vecino))
    dist = train_frec.map(lambda vecino: delta_tfidf(texto_estruct, vecino))

#    k_mas_cercanos = dist.takeOrdered(k, key=lambda x: x[2])
    ## PARA DIST2
    k_mas_cercanos = dist.takeOrdered(k, key=lambda x: -x[0])

    return k_mas_cercanos

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)


reviewRDD = sc.textFile("reviews_test")
review_dataframe = pycsv.csvToDataFrame(sqlCtx, reviewRDD, parseDate=False)
reviews = review_dataframe.rdd

reviews_preprocesado = reviews.map(preprocesar)
reviews = reviews_preprocesado.collect()

cant_ej = 100
test = test_preprocesado.take(cant_ej)
#for t in test:
#    print "t: ", t

## PREDECIR VARIOS
error = 0
correct = 0
for test_pred in test:
    print "TEST_PRED: ", test_pred[0:2]

    k_mas_cercanos = knn(test_pred, 3)
#    print "k_mas_cercanos: ", k_mas_cercanos

    prediccion = predecir(k_mas_cercanos)

    print "PRED: ", prediccion

    if prediccion == test_pred[1]:
        correct += 1
    error += (prediccion - test_pred[1])**2

print " "
error = sqrt(error)
print "ERROR: ", error

print "accuracy: ", ( float(correct)/cant_ej ) * 100


TEST_PRED:  [201541, 3]
PRED:  3.66666666667
TEST_PRED:  [6911, 2]
PRED:  4.0
TEST_PRED:  [461853, 5]
PRED:  4.66666666667
TEST_PRED:  [563294, 5]
PRED:  4.66666666667
TEST_PRED:  [471402, 5]
PRED:  5.0
TEST_PRED:  [139618, 5]
PRED:  2.33333333333
TEST_PRED:  [248946, 5]
PRED:  5.0
TEST_PRED:  [163663, 1]
PRED:  2.33333333333
TEST_PRED:  [140792, 5]
PRED:  5.0
TEST_PRED:  [340744, 1]
PRED:  2.0
TEST_PRED:  [278204, 4]
PRED:  4.33333333333
TEST_PRED:  [165496, 1]
PRED:  2.33333333333
TEST_PRED:  [415793, 5]
PRED:  3.33333333333
TEST_PRED:  [268341, 4]
PRED:  4.33333333333
TEST_PRED:  [371580, 5]
PRED:  4.66666666667
TEST_PRED:  [538393, 4]
PRED:  5.0
TEST_PRED:  [415317, 5]
PRED:  5.0
TEST_PRED:  [524561, 1]
PRED:  2.33333333333
TEST_PRED:  [55501, 3]
PRED:  4.33333333333
TEST_PRED:  [545857, 5]
PRED:  5.0
TEST_PRED:  [64276, 5]
PRED:  4.0
TEST_PRED:  [549022, 5]
PRED:  5.0
TEST_PRED:  [402962, 4]
PRED:  4.66666666667
TEST_PRED:  [514329, 5]
PRED:  1.66666666667
TEST_PRED:  [549413, 4]
